# Download Sentinel-2 NIR band images from AWS and crop

Requires AWS and GDAL command line functionality.

In [1]:
import os
import subprocess
import numpy as np
import shutil
import geopandas as gpd
import fiona
import shapely
import glob
import matplotlib.image as mpimg

# Enable fiona KML file reading driver
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

In [2]:
# set basepath where the images will be downloaded
# basepath = '/Volumes/JUKES_EXT/CautoRIFT_sites/S2_B3/'
basepath = '/Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/images/S2/'

# basepath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/'
# basepath = '/Users/jukesliu/Documents/GOI_Alaska/fieldmap/images/'

# 0) Automatically the identify S2 tile that overlaps your AOI

Must download the S2 tiling grid from here: https://sentiwiki.copernicus.eu/web/s2-products

In [3]:
# AOIpath = '/Volumes/JUKES_EXT/AUTO-TERMINUS-TRACES/Antarctic-test/BoxThwaites/BufferThwaites_PS3031.shp' # path to the AOI rectangle shapefile
# AOIpath = '/Users/jukesliu/Documents/PECLET/B3/B3/B3_Box_WGS.shp' 
AOIpath = '/Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/AOIs/Wolverine-2018-box-WGS.shp' 
# AOIpath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/BoxTurner.shp' 

# set the path to the S2 tiling grid KML file:
# S2grid_path = '/Users/jukesliu/Documents/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'
S2grid_path = '/Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/AOIs/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml'

In [4]:
# SET YEARS AND MONTHS TO DOWNLOAD
years = np.arange(2015,2025); print(years) # set year(s) to download
months = np.linspace(1,12,12); print(months) # set month(s) to download
outputpath = basepath # outputpath will be in the basepath folder

[2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]


In [5]:
## Reproject AOI box to EPSG:4326 if source_srs differs  ####################################
AOI = fiona.open(AOIpath).next()
aoi = AOI['geometry']['coordinates'][0]
print(aoi)

# automatically extract the source coordinate reference system from the input file
from fiona.crs import to_string
with fiona.open(AOIpath) as colxn:
    source_srs_str = to_string(colxn.crs)
source_srs = source_srs_str[6:]
print(source_srs)
# manually enter source_srs if needed
# source_srs = '32606' # EPSG code for the current projection of the glacier shapefile(s)

# Reproject to WGS if in a different srs 
if not source_srs.endswith('4326'):
    print('reprojecting file')
    #IF GENERIC NAME: "BoxID" with number
    # boxespath = basepath+"Box"+BoxID+"/Box"+BoxID # access the BoxID folders created 
    # rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:"+source_srs+" "
    # rp +=boxespath+"_WGS.shp "+boxespath+".shp"
    #IF CUSTOM NAME (NOTE: customized indexing based on input AOI name)
    if source_srs.startswith('epsg'):
        rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs "+source_srs+" "
    else:
        rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:"+source_srs+" "
    rp +=AOIpath[:-7]+"WGS.shp "+AOIpath 
    
    # check the command and run it
    print("Command:", rp) # check command
    subprocess.run(rp, shell=True, check=True) # run the command on terminal
else:
    print('AOI already in EPSG:4326')

# # if an error is produced, check the error output on the terminal window that runs this notebook
######################################################################################

[(-148.94813633153555, 60.37595442278861), (-148.9516668415369, 60.43494807547127), (-148.8584223812696, 60.43628128520052), (-148.85506030588365, 60.37728445544937), (-148.94813633153555, 60.37595442278861)]
epsg:4326
AOI already in EPSG:4326


/var/folders/4r/83h4jj3152b30h9kpnc_jg280000gq/T/ipykernel_32529/458752097.py:2: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  AOI = fiona.open(AOIpath).next()


In [6]:
aoi_gdf = gpd.read_file(AOIpath) # read in the AOI shapefile
aoi = aoi_gdf.geometry.values
aoi

<GeometryArray>
Length: 1, dtype: geometry

In [7]:
s2grid_shp = fiona.open(S2grid_path) # open the S2 tile grid

# find the S2 foorptint tile(s) overlapping the AOI
tilename = 'None'
for feature in s2grid_shp:
    tile = shapely.geometry.Polygon(feature['geometry']['geometries'][0]['coordinates'][0])
#     print(feature['properties']['Name'])
    if tile.overlaps(aoi[0]):
        tilename=feature['properties']['Name']
        print(tilename)
        # break # stop searching
print(tilename)

06VUM
06VUM


In [8]:
# construct the folderpath (one at a time)
tilefolder = str(int(tilename[0:2]))+'/'+tilename[2:3]+'/'+tilename[3:]+'/'
print(tilefolder)

6/V/UM/


## Or set it manually here:

In [ ]:
# tilefolder = '13/C/DS/'
tilefolder = '7/V/EG/'

## Explore files available on AWS manually:

In [ ]:
# # explore files manually:
# !export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws s3 ls s3://sentinel-cogs/sentinel-s2-l2a-cogs/35/X/MH/2024/ --no-sign-request

In [ ]:
# # try filtering by clouds
# from pystac_client import Client
# client = Client.open(api_url)
# collection = "sentinel-2-l2a"

# search = client.search(
#     collections=[collection],
#     MGRS_TILE='6VUM'
#     datetime="2020-03-20/2020-03-30",
#     query=["eo:cloud_cover<50"]
# )
# print(search.matched())

# 1) Download data masks

Syntax:

aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/031/005/ Output/path/LS8aws/Path031_Row005/ --recursive --exclude "*" --include "*MTL.txt"

In [9]:
# Specify the AOI extent (aka box) in UTM coordinates (used for cropping)
# boxpath = AOIpath #  path to the AOI Box (UTM!!)
# imagepath = '/Users/jukesliu/Documents/GOI_Alaska/fieldmap/images/' # path to the downloaded images
# boxpath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/BoxTurner_UTM_07.shp'  #  path to the AOI Box (UTM!!)
boxpath = '/Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/AOIs/Wolverine-2018-box-WGS_UTM_06.shp'   #  path to the AOI Box (UTM!!)

In [10]:
# Download the classification masks and reorganize files (https://sentiwiki.copernicus.eu/web/s2-processing#S2Processing-SceneClassification(SC)S2-Processing-Scene-Classificationtrue)
# NO_DATA = 0, CLOUD_MEDIUM_PROBABILITY = 8, CLOUD_HIGH_PROBABILITY = 9
band = 'SCL'

# loop through and download
for year in years:
    for month in months:
        year = str(year); month = str(int(month)) # convert to strings
        print('Downloading', year, month)

        cmd = 'export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; '
        cmd += 'aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/'+tilefolder+year+'/'+month+'/'
        cmd += ' '+outputpath+' --recursive --exclude "*/*" --include "*/'+band+'.tif"' # change the band here
        print(cmd)
        subprocess.run(cmd, check=True, shell=True)


# loop through the image subdirectories, rename the SCL.tif file with image identifiers, & move out of subdirectories
for imgfolder in glob.glob(basepath+'S2*'):
    if not imgfolder.endswith('.tif') and not imgfolder.endswith('.xml'):
        files = os.listdir(imgfolder)
        for file in files:
            if file == band+'.tif': # rename the files (can change to different band if desired)
                spath = imgfolder+'/'+band+'.tif'
                tpath = basepath+imgfolder.split('/')[-1]+'_'+band+'.tif'
                os.rename(spath, tpath)  
                shutil.rmtree(imgfolder)

# crop SCL tiffs with gdalwarp (command line)
for image in os.listdir(basepath): 
    # if it hasn't already been clipped
    if not os.path.exists(basepath+image[:-4]+band+'_clipped.tif') and image.endswith(band+'.tif'):
        if image.split('_')[2].startswith(('202','201')):
            crop_cmd = 'gdalwarp -cutline '+boxpath+' -crop_to_cutline '+basepath+image+" "+basepath+image[:-4]+'_clipped.tif'
            print(crop_cmd)
            os.system(crop_cmd)

# DELETE THE ORIGINAL FILES AFTER CLIPPING BECAUSE THEY ARE HUGE! (either here or in terminal)
for image in os.listdir(basepath): 
    if not image.endswith(('_clipped.tif','_clipped_10m.tif')):
        os.remove(basepath+image)

export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/6/V/UM/2015/1/ /Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/images/S2/ --recursive --exclude "*/*" --include "*/SCL.tif"
export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/6/V/UM/2015/2/ /Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/images/S2/ --recursive --exclude "*/*" --include "*/SCL.tif"
export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/6/V/UM/2015/3/ /Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/images/S2/ --recursive --exclude "*/*" --include "*/SCL.tif"
export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/6/V/UM/2015/4/ /Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolveri

# 2) Identify files that are sufficiently cloud-free that they are worth downloading

In [11]:
######################################################################################
# Specify the band to download (Sentinel-2 B 8 is NIR)
band = 'B08'

# Adjust cloud thresholds here:
SCLPIXEL_thresh_lower = 8 # medium probability of clouds in Sentinel-2 L2A products
SCLPIXEL_thresh_upper = 9 # high probability of clouds in Sentinel-2 L2A products

cpercent_thresh = 50.0 # maximum cloud cover % in terminus box
fpercent_thresh = 60.0 # maximum NO_DATA % in terminus box
######################################################################################

In [ ]:
#follow the same methodology as used to identify good Landsat images but using the SCL file, not QA band
for image in os.listdir(basepath): 
    print(image[:-16])
    SCLpixel = mpimg.imread(basepath+image) # read in QAPIXEL file as numpy array
    totalpixels = SCLpixel.shape[0]*SCLpixel.shape[1] # count total number of pixels

    cloudSCLpixel = SCLpixel[((SCLpixel >= SCLPIXEL_thresh_lower) & (SCLpixel < SCLPIXEL_thresh_upper))]
    
    # calculate percentages of cloud and fill pixels
    fillpixel = SCLpixel[SCLpixel == 0] 
    cloudpixels = len(cloudSCLpixel); fillpixels = len(fillpixel) # count the cloudy and fill pixels
    cloudpercent = int(float(cloudpixels)/float(totalpixels)*100) # calculate percent cloudy
    fillpercent = int(float(fillpixels)/float(totalpixels)*100) # calculate percent fill

    # evaluate thresholds
    if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
        # download the band for that scene into your scene folders:
        band = str(band) # string format
        year = str(image[9:13]); month = str(image[13:15]) # convert to strings
        # print(year)
        # print(month)

        cmd = 'export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; '
        cmd += 'aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/'+tilefolder+year+'/'+month+'/'
        cmd += ' '+outputpath+' --recursive --exclude "*/*" --include "'+image[:-16]+'/'+band+'.tif"' # change the band here
        print(cmd)
        
        subprocess.run(cmd, check=True, shell=True)
    else:
        print('failed cloud & fill thresholds: Cloud % ', cloudpercent, 'Fill %', fillpercent)
        os.remove(basepath+image)


S2B_6VUM_20200819_0_L2A
2020
08
export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/6/V/UM/2020/08/ /Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/images/S2/ --recursive --exclude "*/*" --include "S2B_6VUM_20200819_0_L2A/B08.tif"
S2A_6VUM_20210308_2_L2A
failed cloud & fill thresholds: Cloud %  0 Fill % 100
S2A_6VUM_20230527_0_L2A
failed cloud & fill thresholds: Cloud %  0 Fill % 100
S2B_6VUM_20230221_0_L2A
failed cloud & fill thresholds: Cloud %  0 Fill % 100
S2A_6VUM_20170525_0_L2A
2017
05
export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://sentinel-cogs/sentinel-s2-l2a-cogs/6/V/UM/2017/05/ /Users/ellynenderlin/Research/NASA_CryoIdaho/glaciers/Wolverine/images/S2/ --recursive --exclude "*/*" --include "S2A_6VUM_20170525_0_L2A/B08.tif"
S2A_6VUM_20211008_1_L2A
2021
10
export PATH=/usr/local/bin:/usr/bin:/bin:/usr/sbin:$PATH; aws --no-sign-request s3 cp s3://s

# 4) Sort & crop images to AOI:

In [ ]:
# set paths
# imagepath = '/Volumes/JUKES_EXT/CautoRIFT_sites/S2_KG/' # path to the downloaded images
# imagepath = '/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/sentinel2/' # path to the downloaded images
imagepath = basepath # path to the downloaded images


In [ ]:
# move out of subdirectories and rename with identifying information
for imgfolder in glob.glob(basepath+'S2*'):
    if not imgfolder.endswith('.tif') and not imgfolder.endswith('.xml'):
        files = os.listdir(imgfolder)
        for file in files:
            if file == band+'.tif': # rename the files (can change to different band if desired)
                spath = imgfolder+'/'+band+'.tif'
                tpath = basepath+imgfolder.split('/')[-1]+'_'+band+'.tif'
                os.rename(spath, tpath)  
                shutil.rmtree(imgfolder)

In [ ]:
# # REPROJECT FIRST: specify the correct UTM zone!!! (326XX for N hemi)
# for image in os.listdir(imagepath): 
#     if not os.path.exists(imagepath+'reprojected/'+image): # if it hasn't already been reprojected
#         rp_cmd = 'gdalwarp -t_srs EPSG:32608 '+imagepath+image+' '+imagepath+'reprojected/'+image
#         os.system(rp_cmd)

In [ ]:
# # crop with gdalwarp (command line) - REPROJECTED IMAGES
# for image in os.listdir(imagepath+'reprojected/'): 
#     # if it hasn't already been clipped
#     if not os.path.exists(imagepath+'reprojected/'+image[:-4]+'_clipped.tif') and image.endswith('.tif'):
#         crop_cmd = 'gdalwarp -cutline '+boxpath+' -crop_to_cutline '+imagepath+'reprojected/'+image+" "+imagepath+'reprojected/'+image[:-4]+'_clipped.tif'
#         print(crop_cmd)
#         os.system(crop_cmd)

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220320__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220307__B08.tif to destination /Volumes/SURGE_DIS

...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220406__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220421__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220421__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220514__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220514__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220514__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220514__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220517__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220517__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220517__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220517__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220517__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220517__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220526__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220526__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220526__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220526__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220526__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220526__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220529__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220529__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220605__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220605__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220605__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220605__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220608__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220608__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220608__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220608__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220608__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20220608__B08_clipped.tif.


...

Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220924__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220918__B08_clipped.tif.


...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221004__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221008__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221008__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221009__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221016__B08.tif to destination /Volumes/SURGE_DIS

...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20221130__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
gdalwarp -cutline /Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/LO/LO_Box_UTM08.shp -crop_to_cutline /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221102__B08.tif /Volumes/SURGE_DISK/S2_LO/S2B_8VLM_20221102__B08_clipped.tif
Creating output file that is 4646P x 3921L.
Processing /Volum

Creating output file that is 4646P x 3921L.
Processing /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08.tif [1/1] : 0Using internal nodata values (e.g. 0) for image /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08.tif.
Copying nodata values from source /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08.tif to destination /Volumes/SURGE_DISK/S2_LO/S2A_8VLM_20220114__B08_clipped.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
# crop with gdalwarp (command line)
for image in os.listdir(imagepath): 
    # if it hasn't already been clipped
    if not os.path.exists(imagepath+image[:-4]+band+'_clipped.tif') and image.endswith(band+'.tif'):
        if image.split('_')[2].startswith(('202','201')):
            crop_cmd = 'gdalwarp -cutline '+boxpath+' -crop_to_cutline '+imagepath+image+" "+imagepath+image[:-4]+'_clipped.tif'
            print(crop_cmd)
            os.system(crop_cmd)

In [ ]:
# DELETE THE ORIGINAL FILES AFTER CLIPPING BECAUSE THEY ARE HUGE! (either here or in terminal)
for image in os.listdir(imagepath): 
    if not image.endswith(('_clipped.tif','_clipped_10m.tif')):
        # print(imagepath+image)
        os.remove(imagepath+image)